In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import os
import warnings
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.layers import (Activation, Dropout, Flatten, Dense, Input, Conv2D, 
                          MaxPooling2D, BatchNormalization, Concatenate,
                          Conv2DTranspose, concatenate)
from keras.models import Sequential, model_from_json
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import Sequence

from numpy import argmax
from PIL import Image
from tqdm import tqdm

warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [98]:
from google.colab import drive
drive.mount('/content/drive')
!pip install --upgrade tables

json_file = open('/content/drive/My Drive/Colab Notebooks/clasificador-animales/model.json', 'r')
json_file = model
json_file.close()
loaded_model = model_from_json(loaded_model_json)
print('Model uploaded!')

loaded_model.load_weights('/content/drive/My Drive/Colab Notebooks/clasificador-animales/weights.h5')
print('Weights uploaded!')

label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('/content/drive/My Drive/Colab Notebooks/clasificador-animales/label_encoder.npy')
print('LabelEncoder uploaded!')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.4.4)
Model uploaded!
Weights uploaded!
LabelEncoder uploaded!


In [89]:
'''
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('classes.npy')
'''

Loaded model from disk


In [0]:
BATCH_SIZE = 32
SHAPE = (128,128,3)

In [0]:
class AnimalGenerator(Sequence):
  
  def __init__(self, paths, labels, batch_size, shape, shuffle = False, use_cache = False):
    self.paths, self.labels = paths, labels
    self.batch_size = batch_size
    self.shape = shape
    self.shuffle = shuffle
    self.use_cache = use_cache
    if use_cache == True:
        self.cache = np.zeros((paths.shape[0], shape[0], shape[1], shape[2]), dtype=np.float16)
        self.is_cached = np.zeros((paths.shape[0]))
    self.on_epoch_end()
    
  def __len__(self):
    return int(np.ceil(len(self.paths)/float(self.batch_size)))
  
  def __getitem__(self, idx):
    indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

    paths = self.paths[indexes]
    X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
    # Generate data
    if self.use_cache == True:
        X = self.cache[indexes]
        for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
            image = self.__load_image(path)
            self.is_cached[indexes[i]] = 1
            self.cache[indexes[i]] = image
            X[i] = image
    else:
        for i, path in enumerate(paths):
            X[i] = self.__load_image(path)

    y = self.labels[indexes]

    return X, y
  
  def __load_image(self, path):
    im = cv2.imread(path)
    im = cv2.resize(im, (SHAPE[0], SHAPE[1]), interpolation=cv2.INTER_CUBIC)
    im = np.divide(im, 255)
    return im
    
  def on_epoch_end(self):
      # Updates indexes after each epoch
      self.indexes = np.arange(len(self.paths))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)

  def __iter__(self):
      """Create a generator that iterate over the Sequence."""
      for item in (self[i] for i in range(len(self))):
          yield item


In [0]:
def translator(x):
    return {
        'cane': 'Perro',
        'cavallo': 'Caballo',
        'elefante': 'Elefante',
        'farfalla': 'Mariposa',
        'gallina': 'Gallina',
        'gatto': 'Gato',
        'mucca': 'Vaca',
        'pecora': 'Oveja',
        'ragno': 'Araña',
        'scoiattolo': 'Ardilla',
    }[x]

In [136]:
paths = ['/content/drive/My Drive/Colab Notebooks/clasificador-animales/image.jpg']
labels = ['']

paths = np.array(paths)
labels = np.array(labels)

animal_gen = AnimalGenerator(paths, labels, BATCH_SIZE, SHAPE, use_cache=True, shuffle = False)

animal_pred = loaded_model.predict_generator(animal_gen)

# invert first example
inverted = label_encoder.inverse_transform([argmax(animal_pred[0])])
animal = inverted[0].decode('UTF-8')

print('Perro:    {:6.2f} %'.format(animal_pred[0][0]*100))
print('Caballo:  {:6.2f} %'.format(animal_pred[0][1]*100))
print('Elefante: {:6.2f} %'.format(animal_pred[0][2]*100))
print('Mariposa: {:6.2f} %'.format(animal_pred[0][3]*100))
print('Gallina:  {:6.2f} %'.format(animal_pred[0][4]*100))
print('Gato:     {:6.2f} %'.format(animal_pred[0][5]*100))
print('Vaca:     {:6.2f} %'.format(animal_pred[0][6]*100))
print('Oveja:    {:6.2f} %'.format(animal_pred[0][7]*100))
print('Araña:    {:6.2f} %'.format(animal_pred[0][8]*100))
print('Ardilla:  {:6.2f} %'.format(animal_pred[0][9]*100))
print('')
print('Predicción: ', translator(animal))

Perro:      5.93 %
Caballo:    0.03 %
Elefante:   0.44 %
Mariposa:   0.08 %
Gallina:    0.63 %
Gato:      82.09 %
Vaca:       0.08 %
Oveja:      0.47 %
Araña:      4.47 %
Ardilla:    5.80 %

Predicción:  Gato
